In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import os
from matplotlib import pyplot as plt
import pandas as pd 

In [3]:
def calculate_angle(a,b,c):
    a=np.array(a) 
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180.0:
        angle=360-angle
    return angle 
def target(path,names):
    for i in names.keys():
        if i in path:
            return names[i]
def split_seq(seq,num_pieces):
    start=0
    for i in range(num_pieces):
        stop=start+len(seq[i::num_pieces])
        yield seq[start:stop]
        start=stop

In [4]:
cap=cv2.VideoCapture('C:\collez_stady\FitnForm\push up\side-model/arch-back/26.mp4')
pushing=False
chk=[150,150]
iter=0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        if ret:
            image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            results=pose.process(image)
            image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
            try:
                landmarks=results.pose_landmarks.landmark
                wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] 
                elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee=[landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                angle1=calculate_angle(wrist,elbow,shoulder)
                angle2=calculate_angle(knee,hip,shoulder)
                cv2.putText(image,str(angle1),tuple(np.multiply(elbow,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255, 255),2,cv2.LINE_AA)
                cv2.putText(image,str(angle2),tuple(np.multiply(hip,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255, 255),2,cv2.LINE_AA)
                #print(angle1)
                if iter<20:
                    chk[0]=(chk[0]+angle1)/2
                    chk[1]=(chk[1]+angle2)/2
                    iter+=1
                elif iter==20:
                    if (chk[0]>=155 and chk[0]<=180) and (chk[1]>=155 and chk[1]<=180):
                        pushing=True
                    iter=0
                cv2.putText(image,str(angle2),tuple(np.multiply(hip,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255, 255),2,cv2.LINE_AA)
            except:
                pass
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))               
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            if pushing:
                cv2.putText(image,'Pushing',(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255, 255),2,cv2.LINE_AA)
                #arr.append(angle2)
            else:
                cv2.putText(image,'Not Pushing',(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            cv2.imshow('Feed',image)
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [1]:
import pyttsx3
pyttsx3.speak("Let's Go, Push ups baby")
print(3)

3
